Создать локальную векторную базу данных Milvus

In [56]:
from pymilvus import MilvusClient

client = MilvusClient("milvus_demo.db")

Создание коллекции

In [82]:
if client.has_collection(collection_name="ru_sbert_demo_collection"):
    client.drop_collection(collection_name="ru_sbert_demo_collection")
client.create_collection(
    collection_name="ru_sbert_demo_collection",
    dimension=1024,  # The vectors we will use in this demo has 768 dimensions
)

client.describe_collection("ru_sbert_demo_collection")

{'collection_name': 'ru_sbert_demo_collection',
 'auto_id': False,
 'num_shards': 0,
 'description': '',
 'fields': [{'field_id': 100,
   'name': 'id',
   'description': '',
   'type': <DataType.INT64: 5>,
   'params': {},
   'is_primary': True},
  {'field_id': 101,
   'name': 'vector',
   'description': '',
   'type': <DataType.FLOAT_VECTOR: 101>,
   'params': {'dim': 1024}}],
 'functions': [],
 'aliases': [],
 'collection_id': 0,
 'consistency_level': 0,
 'properties': {},
 'num_partitions': 0,
 'enable_dynamic_field': True}

## Подготовка данных

Сгенерируйте векторные вкрапления с помощью модели по умолчанию. Milvus ожидает, что данные будут представлены в виде списка словарей, где каждый словарь представляет запись данных, называемую сущностью.

In [58]:
from sentence_transformers import SentenceTransformer

# Загружаем модель (скачивается автоматически при первом запуске)
model = SentenceTransformer("ai-forever/sbert_large_nlu_ru")

In [ ]:
# Тексты для векторизации
docs = [
    "Я работаю в АБК.",
    "АБК это айти компания - профессиональное коллекторское агентство",
    "моя должность ведущий разработчик",
    "я разрабатываю проекты на языке python",
    "я использую наушники фирмы Sony"
]

# Получаем эмбеддинги (вектора)
vectors = model.encode(docs, normalize_embeddings=True)

print("Dim:", vectors[0].shape)  # Dim: 768 (768,)

# # Each entity has id, vector representation, raw text, and a subject label that we use
# # to demo metadata filtering later.
data = [
    {"id": i, "vector": vectors[i], "text": docs[i], "subject": "about me"}
    for i in range(len(vectors))
]

print("Data has", len(data), "entities, each with fields: ", data[0].keys())
print("Vector dim:", len(data[0]["vector"]))

Dim: (1024,)
Data has 5 entities, each with fields:  dict_keys(['id', 'vector', 'text', 'subject'])
Vector dim: 1024


Вставка данных

In [80]:
res = client.insert(collection_name="ru_sbert_demo_collection", data=data)

print(res)

{'insert_count': 5, 'ids': [0, 1, 2, 3, 4]}


Векторный поиск

In [81]:
query_list = ["в чем я слушаю музыку", "как я воспроизвожу подкасты"]
query_vectors = model.encode(query_list)
# If you don't have the embedding function you can use a fake vector to finish the demo:
# query_vectors = [ [ random.uniform(-1, 1) for _ in range(768) ] ]

res = client.search(
    collection_name="ru_sbert_demo_collection",  # target collection
    data=query_vectors,  # query vectors
    limit=2,  # number of returned entities
    output_fields=["text", "subject"],  # specifies fields to be returned
)

# print(res)

for i, query_result in enumerate(res):
    for line in query_result:
        print(f"{query_list[i]}?:\t {line['entity']['text']}\t conf = {line['distance']:.2f}")

в чем я слушаю музыку?:	 я использую наушники фирмы Sony	 conf = 0.46
как я воспроизвожу подкасты?:	 я использую наушники фирмы Sony	 conf = 0.50
